In [1]:
import toml
import os
from IOHandlers.carmaker import CarMakerHandler
from Discretizers.fastdiscretizer import FastDiscretizer
from Runners.carmaker import CarMakerRunner
from PngHandlers.yolo import yoloHandler
from ScoreCalculators.iou import IouScoreCalculator
from Evaluation.GCI import GCIEvaluater
import numpy as np 

ldf is not supported


**testrun** -> CM -> **PNG** -> YOLOv5 -> **labels(xxx.txt)**

**testrun** -> CM -> **GT** -> **Metadata(xxx.toml)**

**PNG** -> YOLOv5 -> **labels(xxx.txt)**

In [8]:
for i in range(1,6):
    png_path = r"C:\Arbeit\parameter-discretization\py\Final_results\determinismus\Yolov5_samepng\exp" + str(i) + r"\labels\test_distance.0.txt"
    with open(png_path) as ff:
        label = ff.read()
    if i > 1:
        print(label_old == label)
    label_old = label

True
True
True
True


**testrun** -> CM -> **PNG**

In [28]:
from PIL import Image
from PIL import ImageChops
def compare_images(img1, img2):
    im1 = Image.open(img1).convert('RGB')
    im2 = Image.open(img2).convert('RGB')
    diff = ImageChops.difference(im1, im2)
    pix = diff.load()
    pixel_value = {}
    #diff.show()
    #pix1 = im1.load()
    #pix2 = im2.load()
    row, column = diff.size
    for i in range(row):
        for j in range(column):
            if pix[i,j] != (0,0,0):
                pixel_value[(i,j)] = pix[i,j]
    return pixel_value 

In [30]:
images_results = []
for i in range(1,5):
    path1 = r"C:\Arbeit\parameter-discretization\py\Final_results\determinismus\CM_sametestrun\movie" + str(i) + r".png"
    path2 = r"C:\Arbeit\parameter-discretization\py\Final_results\determinismus\CM_sametestrun\movie" + str(i+1) + r".png"
    images_results.append(compare_images(path1, path2))

In [45]:
images_results[3]

{(64, 285): (1, 1, 1),
 (65, 286): (1, 1, 1),
 (66, 285): (1, 0, 0),
 (66, 286): (0, 1, 0),
 (67, 285): (1, 1, 0),
 (79, 264): (0, 1, 0),
 (118, 254): (1, 1, 1),
 (603, 275): (0, 0, 1),
 (760, 265): (1, 1, 0),
 (761, 265): (1, 1, 0),
 (761, 266): (0, 0, 1),
 (763, 265): (0, 1, 0)}

**testrun** -> CM -> **PNG** -> YOLOv5 -> **labels(xxx.txt)**

**testrun** -> CM -> **GT** -> **Metadata(xxx.toml)**

In [2]:
runner = CarMakerRunner(executable_path=r"C:\IPG\carmaker\win64-10.0\bin\CM.exe", convert_results=True, log_level=1)
out_quants = ['Traffic.T00.tx', 'Traffic.T00.ty', 'Traffic.T00.tz']
camera_name = 'Dev_Xu_frontcar'
para_range = [50,51]
for i in range(10):
    input_handler = CarMakerHandler(r"C:\Arbeit\Carmaker_test\Data\TestRun\determinismus\test" + str(i)+ r"\test_distance", 'Kreuzung_T_TTC.param')
    _, grid_path = FastDiscretizer.generate_instances(input_handler, 1)
    runner.simulate_movies(grid_path, out_quants, camera_name)
    yoloHandler.evaluate_pngs(grid_path)
    IouScoreCalculator.parse_results(grid_path, out_quants, camera_name)

In [56]:
determin = []
sc01 = []
sc02 = []
GT01 = []
GT02 = []
for i in range(9):
    path01 = r"C:\Arbeit\Carmaker_test\Data\TestRun\determinismus\test" + str(i) + "\instances\[50.0].toml"
    path02 = r"C:\Arbeit\Carmaker_test\Data\TestRun\determinismus\test" + str(i) + "\instances\[51.0].toml"
    with open(path01) as ff:
        result01 = toml.load(ff)
        sc01.append(result01['results']['safetyscore'])
        GT01.append((result01['results']['ipgresult']["Traffic.T00.tx"], result01['results']['ipgresult']["Traffic.T00.ty"],result01['results']['ipgresult']["Traffic.T00.tz"]))
    with open(path02) as ff:
        result02 = toml.load(ff)
        sc02.append(result02['results']['safetyscore'])
        GT02.append((result02['results']['ipgresult']["Traffic.T00.tx"], result02['results']['ipgresult']["Traffic.T00.ty"],result02['results']['ipgresult']["Traffic.T00.tz"]))
GT_jug01 = [GT01[0] == i for i in GT01] 
GT_jug02 = [GT02[0] == i for i in GT02] 
print(GT_jug01,'\n', GT_jug02,'\n', sc01,'\n', sc02,'\n')  

[True, True, True, True, True, True, True, True, True] 
 [True, True, True, True, True, True, True, True, True] 
 ['0.6427083132951259', '0.6427026188077889', '0.6427050593023619', '0.6427050593023619', '0.6427091267933169', '0.6427034323059799', '0.6427058728005529', '0.6427026188077889', '0.6427083132951259'] 
 ['0.6568253589044447', '0.6568182781455251', '0.6568129675763354', '0.6568173930506602', '0.6568173930506602', '0.6568235887147149', '0.6568156228609302', '0.6568173930506602', '0.6568120824814704'] 

